# Extracción características MODIFICADAS Disvoice
## Adrián Arnaiz
## Tareas del notebook
* Extraacción medidas Disvoice para todos los audios **+ atributos edad y sexo**.
 * *Ruta donde guardar: CaracteristicasExtraidas/EdadYSexo*
 
 
* Extracción medidas Disvoice para audios **separados por sexo + atributo edad**.
 * *Ruta donde guardar: CaracteristicasExtraidas/DivisionSexo, -/mujeres, -/hombres*
 
 
* Extracción medidas Disvoice **EXACTAS a Orozco 2016**.
  * *Ruta donde guardar: CaracteristicasExtraidas/Orozco2016, -/onset, -/offset*
  * 12 MFCC y 22 BBE para transiciones Unvoiced (offset y onset separados)
  
----------------
<a id="index"></a>
## Índice del notebook

1. [Introducción](#intro)
    - [Crear Diccionario de informacion de audios](#dic)
    - [Crear clase ExtraccionCaracteristicas](#clase)
2. [Medidas más edad y sexo](#edsex)  
3. [Medidas separadas por sexo más edad](#sexo)
4. [Medidas exactas a Orozco 2016: 12 MFCC y 22 BBE](#orzco)
----------------
  
## 1. Introducción <a id="intro"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>
Al terminar este notebook tendremos un total de **106 conjuntos diferentes con los que hacer experimentos**, de los cuales ya hemos hecho experimentos con 18 de ellos. Las cuales tienen esta estructura en nuestro proyecto:

`CaracterísticasExtraidas.
|
|   Npy: Ccas art, fon y prs directas de disvoice (sin edad ni sexo) [18 sets] -> audios (100-300)
|   
+---DivisionSexo
|   +---hombres
|   |	    Npy: Ccas art, art_on/off, fon y prs de hombres + EDAD Y SEXO [ 30 sets :(18 + 12 {6on+6off}) ] -> audios (50-150)	
|   |           
|   \---mujeres
|           Npy: Ccas art, art_on/off, fon y prs de mujeres + EDAD Y SEXO [ 30 sets :(18 + 12 {6on+6off}) ] -> audios (50-150)      
|           
+---EdadYSexo
|       Npy: CCas art, fon y prs directas de disvoice + EDAD Y SEXO [18 sets] -> audios (100-300)     
|       
\---Orozco2016
    +---offset
    |       Npy: Ccas art_offset + EDAD Y SEXO [ 6 sets ] -> audios (100-300) 
    |       
    \---onset
            Npy: Ccas art_onset + EDAD Y SEXO [ 6 sets ] -> audios (100-300)`
`

### 1.1 Creamos el diccionario con la información de los audios <a id="dic"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>
Guardamos para cada audio el valor del sexo y la edad. También guardamos el nivel de severidad del Parkinson con la escala UPDRS para cada audio para ser utilizado en futuros experimentos
* **Claves**: nombre del audio
* **Valor**: Diccionario que contiene *UPDRS*, *AGE* y *SEX* (0$\rightarrow$M, 1$\rightarrow$F)
> `{'AVPEPUDEA0001': {'UPDRS': 28, 'SEX': 0, 'AGE': 64},..., 'AVPEPUDEAC0057': {'UPDRS': 0, 'SEX': 1, 'AGE': 50}}`

In [1]:
from extractorCcas import ExtractorCaracteristicas
import pickle #Para guardar diccionario de información de audios
import numpy as np
import pandas as pd
import os
from IPython.display import clear_output

In [2]:
#Creamos el diccionario con la estructura indicada
filepath = 'PC-GITA/PCGITA_metadata.csv'
dic_audios_inf = dict()
with open(filepath) as fp:
    cnt = 0
    for line in fp:
        line = line.split(';')[:8]
        if not line[0].startswith('AVP'):
            line[0]=line[0][3:]
        dic_audios_inf[line[0]]=dict()
        dic_audios_inf[line[0]]['UPDRS'] = 0 if line[1]=='' else int(line[1])
        dic_audios_inf[line[0]]['SEX'] = 0 if line[4]=='M' else 1
        dic_audios_inf[line[0]]['AGE'] = int(line[5])
fp.close()

In [3]:
#Guardamos el diciconario para usarlo en futuras ocasiones sin volver a construirlo
pickle_out = open("./CaracteristicasExtraidas/dict_audios_inf.pickle","wb")
pickle.dump(dic_audios_inf, pickle_out)
pickle_out.close()

In [4]:
#recuperamos el diccionario para confirmar que está guardado correctamente
pickle_in = open("./CaracteristicasExtraidas/dict_audios_inf.pickle","rb")
recover_dic = pickle.load(pickle_in)
recover_dic==dic_audios_inf

True

### 1.2 Clase ExtractorCaracterísticas <a id="clase"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>
Hemos creado la clase **ExtractorCaracterísticas** en el módulo extractorCcas que contiene **métodos** útilies para la extracción de características con Disvoice. Incluye métodos para añadir atributos extra. Estos atributos extra pueden ser añadidos a la vez que se extraen las demás características o pueden ser añadidos posteriormente a las correspondientes matrices numpy (utilizando add_atribs). Útil esto último si ya tenemos las ccas de Disvoice sacadas para no tener que volver a sacarlas. En resumen puede realizar las siguientes tareas:
1. Extracción ccas Disvoice sin atributos Extra
2. Extracción ccas Disvoice con atributos Extra
3. Adicción atributos extra a ccas ya extraídas

**Métodos:**
> * **Constructor**: se pasa como argumentos:
  * la ruta donde se encuentran los audios (donde se encuentran las subcarpetas pd y hc)
  * la ruta donde guardar los archivos de características generados por Disvoice,
  * opcionalmente el diccionario donde se tienen atributos extra de cada audio (edad,sexo...)
* **extraccion_ccas_directorio**: Extrae las características de los audios en la ruta pasada en el constructor. tiene como parámetros:
  * Script Disvoice usado para extraer las ccas [articulation, phonatio, prosody].
  * Nombre del archivo donde guardar las ccas de los pacientes HC.
  * Nombre del archivo donde guardar las ccas de los pacientes PD.
  * Atributos_Extra (list)[opcional]: atributos a añadir a las ccas que deben estar incluidos en el diccionario pasado en el constructor. Si no se pasa nada, no se incluye ningun atributo extra.
* **add_atribs**: Añade los atributos a la matriz de ccas pasada, cuyos nombres se pasan en formato string y cuyo valor se encuentra en el diccionario pasado en el constructor. Éste método es útil para añadir los nuevos atributos a ccas ya extraídas. En ese caso, no es necesario que la ruta de los audios pasada en el constructor coincida con la matriz numpy que le pasamos, ya que no es usada. Únicamente añadimos las columnas necesarias a la matriz pasada.
  * ccas: matriz a la que añadir los atributos.
  * atribs: lista que contiene los atributos a añadir. Contiene el nombre de los atributos que se debe corresponder con el nombre que tienen en el diccionario pasado en el constructor.
           

#### Ejemplo de uso
**Extracción ccas Disvoice sin atributos Extra**

In [5]:
%%time
extractor = ExtractorCaracteristicas('PC-GITA/read-text/','CaracteristicasExtraidas/PruebaClase/')
fon_rt_ccas_prueba = extractor.extraccion_ccas_directorio('phonation', 'fon_rt_hc.txt' , 'fon_rt_pd.txt')
print(fon_rt_ccas_prueba.shape)

(100, 30)
Wall time: 2min 49s


**Extracción ccas Disvoice con atributos Extra**

In [6]:
%%time
#Añadimos edad y sexo
extractor = ExtractorCaracteristicas('PC-GITA/read-text/','CaracteristicasExtraidas/PruebaClase/', dic_audios_inf)
fon_rt_ccas_prueba = extractor.extraccion_ccas_directorio('phonation', 'fon_rt_hc.txt' , 'fon_rt_pd.txt' , ['AGE','SEX'])
print(fon_rt_ccas_prueba.shape)

Directorio de características ya existente, no se crea nuevo.
(100, 32)
Wall time: 2min 38s


**Adicción atributos extra a ccas ya extraídas**

In [7]:
fon_v_U_pruebas = np.load('CaracteristicasExtraidas/fon_v_U_ccas.npy')
print(fon_v_U_pruebas.shape)

(300, 30)


In [8]:
%%time
extractor = ExtractorCaracteristicas('DaIgualYaEstanExtraidas', 'CaracteristicasExtraidas/PruebaClase/', dic_audios_inf)
fon_v_U_pruebas = extractor.add_atribs(fon_v_U_pruebas, ['AGE'])
print(fon_v_U_pruebas.shape)

Directorio de características ya existente, no se crea nuevo.
(300, 31)
Wall time: 1.99 ms


In [9]:
!rmdir CaracteristicasExtraidas\PruebaClase /S /Q

------------------
## 2. Disvoice + Edad y Sexo <a id="edsex"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>
Mismas características obtenidas que en los primeros experimentos, pero añadiendo atributos de edad y sexo para cada instancia.

Con estas características se repetiran los primeros experimentos para ver si hay mejora. También se repetiraan esos mismos experimentos pero se realizará una validación cruzada estratificada por edad y sexo para mejorar los resultados.

Como hemos comentado tenemos 2 posibilidades para añadir atributos con nuestra clase ExtractorCaracteristicas:
1. Sacar nuevamente las ccas de Disvocie indicando que añada edad y sexo.
2. Utilizar las ccas ya extraídas que están guardadas en archivos numpy en _/src/CaracteristicasExtraidas_ y añadirlas los atributos correspondientes. (Más rapido, no repite el proceso)

**Añadimos para los archivos que no contenian Nan**

In [10]:
!rmdir CaracteristicasExtraidas\EdadYSexo /S /Q

El sistema no puede encontrar el archivo especificado.


In [11]:
rutaCcas = 'CaracteristicasExtraidas/EdadYSexo/'

In [12]:
extractor = ExtractorCaracteristicas('DaIgualYaEstanExtraidas', rutaCcas , dic_audios_inf)

sets_ccas = [d for d in os.listdir('CaracteristicasExtraidas') if d.endswith('.npy')]
for st in sets_ccas:
    #la fonacion para las palabras tienen instancias ruidosas eliminadas, no funcionará reusarlo
    if not st.startswith('fon_w_'): 
        ccasDisv = np.load('CaracteristicasExtraidas/'+st)
        ccasDsvEdadSexo = extractor.add_atribs(ccasDisv, ['AGE', 'SEX'])
        assert ccasDisv.shape[1]+2 == ccasDsvEdadSexo.shape[1]
        np.save(rutaCcas+st, ccasDsvEdadSexo)

**Extraemos nuevamente añadiendo los atributos para los que contenian Nan**

In [13]:
def phonation_word_extraction(palabras):
    '''
    Llamamos a la función de extracción de características con las rutas necesarias
    '''
    ccas_palabras = dict()
    for p in palabras:
        extractor = ExtractorCaracteristicas('PC-GITA/words/'+p+'/','CaracteristicasExtraidas/EdadYSexo/', dic_audios_inf)
        ccas_palabras[p]= extractor.extraccion_ccas_directorio('phonation', 'fon_w_'+p+'_hc.txt' , 'fon_w_'+p+'_pd.txt', ['AGE','SEX'] )
        clear_output()
        print('Palabras analizadas: ',ccas_palabras.keys())
    return ccas_palabras

In [14]:
words=['atleta','campana','gato','petaka','braso']
fon_words_ccas = phonation_word_extraction(words)
for k in fon_words_ccas:
    np.save(rutaCcas+'fon_w_'+k+'_ccas',fon_words_ccas[k])

Palabras analizadas:  dict_keys(['atleta', 'campana', 'gato', 'petaka', 'braso'])


---------------------
## 3. Disvoice separado por sexo + edad <a id="sexo"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>
**Filtramos los obtenidos en vez de extraer nuevos**.
En vez de sacar de nuevo las características, para cada conjunto de datos que acabamos de obtener, dividiremos cada conjunto de datos en 2: uno de los hombres y otro de las mujeres. Lo realizaremos separando las instancias que tienen 0 de las que tienen 1 de la columna del sexo. Posteriormente borraremos esa columna de sexo ya que no nos aporta nueva información.

In [15]:
!rmdir CaracteristicasExtraidas\DivisionSexo /S /Q

El sistema no puede encontrar el archivo especificado.


Creamos carpetas para cada grupo: hombre-mujer

In [16]:
rutaCcas2 = 'CaracteristicasExtraidas/DivisionSexo/'
os.mkdir(rutaCcas2)
rutah = rutaCcas2+'hombres/'
os.mkdir(rutah)
rutam = rutaCcas2+'mujeres/'
os.mkdir(rutam)

Cogemos las características que hemos sacado en el anterior apartado: incluyen sexo y edad.
Primero las dividimos por sexo y eliminamos esa columna ya que no aporta informacion al conjunto.

In [17]:
#Cogemos sets de caracteristicas que contienen edad y sexo
sets_ccas = [d for d in os.listdir('CaracteristicasExtraidas/EdadYSexo') if d.endswith('.npy')]
for conjunto in sets_ccas:
    st = np.load(rutaCcas+conjunto)
    #nos quedamos solo con las mujeres
    mujeres = st[st[:,-2]==1]
    #Borramos la columna sexo ya que no aporta nueva información
    mujeres = np.delete(mujeres, -2, axis=1)
    np.save(rutam+conjunto, mujeres)
    
    hombres = st[st[:,-2]==0]
    hombres = np.delete(hombres, -2, axis=1)
    np.save(rutah+conjunto, hombres)

Comprobamos que se ha realizado la separación de sexos de manera correcta

In [18]:
h = np.load(rutah+sets_ccas[5])
m = np.load(rutam+sets_ccas[5])
hm = np.load(rutaCcas+sets_ccas[5])
assert np.equal(h[:,:-1], hm[hm[:,-2]==0][:,:-2]).all()
assert np.equal(m[:,:-1], hm[hm[:,-2]==1][:,:-2]).all()

----------------
## 4. Extracción de medidas  EXACTAS a Orozco 2016. <a id="orzco"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>
* 12 MFCC y 22 BBE para transiciones Unvoiced (offset y onset separados)
* **Filtramos los obtenidos en vez de extraer nuevos**.

Estas medidas han sido sacadas con el módulo **articulation.py**. Este script nos sacaba 488 características de diferentes medidas de MFCC y BBE en offset y onset y sus derivadas. Sin embargo, los resultados en el primer experimento de clasificación han sido muy negativos. Por ello, replicaremos exactamente las ccas de Orozco.

Elegiremos el subconjunto de ccas de 12 MFCC y 22 BBE para cada conjunto. Por cada grupo de ccas de articulación que tenemos (6: 5 palabras y 1 frase), sacaremos 2: 12MFCC y 22BBE para transición onset y lo mismo para transición offset.

En vez de extraer otra vez las medidas, obtendremos elsubconjunto de las medidas ya extraídas con edad y sexo. Por lo tanto, serán los sets nombrados como art_...npy dentro de la ruta CaracteristicasExtraidas/EdadySexo/ y los resultantes serán guardados en CaracteristicasExtraidas/Orozco2016/onset/ y CaracteristicasExtraidas/Orozco2016/offset.



In [19]:
!rmdir CaracteristicasExtraidas\Orozco2016 /S /Q

El sistema no puede encontrar el archivo especificado.


In [20]:
rutaCcas3 = 'CaracteristicasExtraidas/Orozco2016/'
os.mkdir(rutaCcas3)
rutaonset  = rutaCcas3+'onset/'
os.mkdir(rutaonset)
rutaoffset = rutaCcas3+'offset/'
os.mkdir(rutaoffset)

In [21]:
set_ccas_art = [d for d in os.listdir('CaracteristicasExtraidas/EdadYSexo') if d.endswith('.npy') and d.startswith('art')]
set_ccas_art

['art_rt_ccas.npy',
 'art_w_atleta_ccas.npy',
 'art_w_braso_ccas.npy',
 'art_w_campana_ccas.npy',
 'art_w_gato_ccas.npy',
 'art_w_petaka_ccas.npy']

* Para las ccas en transiciones **onset**, según se indica en la página de github de $\href{https://github.com/jcvasquezc/DisVoice/tree/master/articulation}{Disvoice/Articulation}$, las 12 MFCC y 22 BBE de onset se encuentran en las **posiciones 1-34** (0-33 en un array).
* Para las ccas en transiciones **offset**, las 12 MFCC y 22 BBE de onset se encuentran en las **posiciones 59-92** (58-91 en un array).
* Por lo tanto tendremos que coger esas columnas determinadas más las 3 columnas finales de edad, sexo y PD/HD

In [22]:
for st_art in set_ccas_art:
    st = np.load(rutaCcas+st_art)
    sex_age_target = st[:,-3:]
    ccas_onset = st[:,:34]
    ccas_onset = np.hstack((ccas_onset,sex_age_target))
    ccas_offset = st[:,58:92]
    ccas_offset = np.hstack((ccas_offset,sex_age_target))
    np.save(rutaonset+st_art, ccas_onset)
    np.save(rutaoffset+st_art, ccas_offset)

Ya están extraídas, adicionalmente **las añadimos al directorio anterior separandola por sexos**, así podremos hacer el experimento para este conjunto de ccas con los dos sexos juntos y separados al igual que con los demás conjuntos.

In [23]:
for conjunto in set_ccas_art:
    st_on = np.load(rutaCcas3+'onset/'+conjunto)
    st_of = np.load(rutaCcas3+'offset/'+conjunto)
    #mujeres en onsset
    mujeres_on = st_on[st_on[:,-2]==1]
    #Borramos la columna sexo ya que no aporta nueva información
    mujeres_on = np.delete(mujeres_on, -2, axis=1)
    np.save(rutam+conjunto[:-4]+'_onset', mujeres_on)
    
    hombres_on = st_on[st_on[:,-2]==0]
    hombres_on = np.delete(hombres_on, -2, axis=1)
    np.save(rutah+conjunto[:-4]+'_onset', hombres_on)
    
    mujeres_of = st_of[st_of[:,-2]==1]
    #Borramos la columna sexo ya que no aporta nueva información
    mujeres_of = np.delete(mujeres_of, -2, axis=1)
    np.save(rutam+conjunto[:-4]+'_ofset', mujeres_of)
    
    hombres_of = st_of[st_of[:,-2]==0]
    hombres_of = np.delete(hombres_of, -2, axis=1)
    np.save(rutah+conjunto[:-4]+'_ofset', hombres_of)

In [24]:
def separasexos(rutaOrigen, rutaDestino):
    sets_ccas = [d for d in os.listdir(rutaOrigen) if d.endswith('.npy')]
    for conjunto in sets_ccas:
        st = np.load(rutaOrigen+conjunto)
        #nos quedamos solo con las mujeres
        mujeres = st[st[:,-2]==1]
        #Borramos la columna sexo ya que no aporta nueva información
        mujeres = np.delete(mujeres, -2, axis=1)
        np.save(rutaDestino+'mujeres/'+conjunto, mujeres)

        hombres = st[st[:,-2]==0]
        hombres = np.delete(hombres, -2, axis=1)
        np.save(rutaDestino+'hombres/'+conjunto, hombres)